# **NPA MODEL**

### **Initial Setup**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
route = '/content/drive/MyDrive/IMPACT PROJECT'

%cd {route}/recommenders

/content/drive/MyDrive/IMPACT PROJECT/recommenders


In [3]:
!pip install scrapbook retrying

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.2 MB/s eta 0:00:00


###  **Importing the needed libraries**

In [4]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.npa import NPAModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]
Tensorflow version: 2.12.0


### **Loading the behaviours and news dataframes**

In [7]:
MIND_type = "demo"

In [8]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'npa.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)

if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|██████████| 17.0k/17.0k [00:01<00:00, 10.7kKB/s]
100%|██████████| 9.84k/9.84k [00:01<00:00, 7.62kKB/s]
100%|██████████| 95.0k/95.0k [00:03<00:00, 25.9kKB/s]


---------------

## **NPA: Neural News Recommendation with Personalized Attention**

Setting up the parameters

In [9]:
epochs = 5
seed = 42
batch_size = 32

In [10]:
hparams = prepare_hparams(yaml_file,
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file,
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

HParams object with values {'support_quick_scoring': False, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 100, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 5, 'batch_size': 32, 'show_step': 100000, 'title_size': 10, 'his_size': 50, 'data_format': 'news', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'npa', 'loss': 'cross_entropy_loss', 'wordEmb_file': '/tmp/tmp7fn1n4ko/utils/embedding.npy', 'wordDict_file': '/tmp/tmp7fn1n4ko/utils/word_dict.pkl', 'userDict_file': '/tmp/tmp7fn1n4ko/utils/uid2index.pkl'}


Instantiating the model

In [11]:
iterator = MINDIterator

In [12]:
model = NPAModel(hparams, iterator, seed=seed)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training the NPA model

In [13]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

1086it [00:55, 19.60it/s]
0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
8874it [00:26, 336.13it/s]


at epoch 1
train info: logloss loss:1.521140100547622
eval info: group_auc:0.5737, mean_mrr:0.2485, ndcg@10:0.3337, ndcg@5:0.2661
at epoch 1 , train time: 55.4 eval time: 36.4


1086it [00:43, 24.94it/s]
8874it [00:25, 354.13it/s]


at epoch 2
train info: logloss loss:1.4106470508909137
eval info: group_auc:0.5884, mean_mrr:0.2638, ndcg@10:0.3516, ndcg@5:0.287
at epoch 2 , train time: 43.5 eval time: 35.1


1086it [00:43, 24.95it/s]
8874it [00:25, 351.62it/s]


at epoch 3
train info: logloss loss:1.3470714389729017
eval info: group_auc:0.592, mean_mrr:0.2714, ndcg@10:0.3592, ndcg@5:0.2918
at epoch 3 , train time: 43.5 eval time: 35.3


1086it [00:43, 24.91it/s]
8874it [00:25, 353.21it/s]


at epoch 4
train info: logloss loss:1.297705107191869
eval info: group_auc:0.5921, mean_mrr:0.271, ndcg@10:0.3582, ndcg@5:0.2907
at epoch 4 , train time: 43.6 eval time: 35.0


1086it [00:43, 24.92it/s]
8874it [00:25, 354.32it/s]


at epoch 5
train info: logloss loss:1.2544619664412615
eval info: group_auc:0.5968, mean_mrr:0.2718, ndcg@10:0.3601, ndcg@5:0.2926
at epoch 5 , train time: 43.6 eval time: 35.1
CPU times: user 6min 24s, sys: 42 s, total: 7min 6s
Wall time: 6min 46s


Evaluating the NPA Model

In [14]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

8874it [00:25, 352.52it/s]


{'group_auc': 0.5968, 'mean_mrr': 0.2718, 'ndcg@5': 0.2926, 'ndcg@10': 0.3601}
CPU times: user 51.3 s, sys: 5.03 s, total: 56.4 s
Wall time: 35.3 s


| Model   | group_auc | mean_mrr | ndcg@5 | ndcg@10 |
|----------|-----------|----------|--------|---------|
| NPA    |   0.5968  |   0.2718  | 0.2926 |  0.3601 |